### Системы управления базами данных MongoDB и SQLite в Python

#### Задание 1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД (без датафрейма)

In [1]:
from pymongo import MongoClient
from pymongo import errors
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests as rq
import re

In [2]:
client = MongoClient('localhost',27017)
db = client['jobs_db']

In [3]:
jobs = db.jobs

#### HH

In [4]:
main_link_hh = 'https://spb.hh.ru'
header_hh = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.109 Safari/537.36',
          'Accept':'*/*'}
params_hh = {'clusters':True,
          'enable_snippets':True,
          'text':'Data scientist',
          'L_save_area':True,
          'area':113,
          'from':'cluster_area',
          'showClusters':True}

In [5]:
response_hh = rq.get(main_link_hh + '/search/vacancy', headers=header_hh, params=params_hh).text
soup_hh = bs(response_hh, 'lxml')
vacancy_block_hh = soup_hh.find('div',{'class':'vacancy-serp'})
vacancy_list_hh = vacancy_block_hh.find_all('div',{'class':'vacancy-serp-item'})

In [6]:
pages_hh = []
while True:
    if soup_hh.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'}):
        next_page_hh = main_link_hh + soup_hh.find('a',{'class':'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})['href']
    else:
        break
    pages_hh.append(next_page_hh)
    response_hh = rq.get(next_page_hh, headers=header_hh).text
    soup_hh = bs(response_hh, 'lxml')
    vacancy_block_hh_ = soup_hh.find('div',{'class':'vacancy-serp'})
    vacancy_list_hh_ = vacancy_block_hh_.find_all('div',{'class':'vacancy-serp-item'})
pprint(pages_hh)

['https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=113&clusters=true&enable_snippets=true&text=Data+scientist&page=1',
 'https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=113&clusters=true&enable_snippets=true&text=Data+scientist&page=2',
 'https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=113&clusters=true&enable_snippets=true&text=Data+scientist&page=3',
 'https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=113&clusters=true&enable_snippets=true&text=Data+scientist&page=4',
 'https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=113&clusters=true&enable_snippets=true&text=Data+scientist&page=5']


In [7]:
jobs_hh = []

In [8]:
# Функция добавления вакансий HH в базу данных

def add_jobs_hh_to_db(jobs_hh):
    
    jobs.insert_many(jobs_hh)

    return True

In [9]:
for job_hh in vacancy_list_hh + vacancy_list_hh_:
    
    job_data_hh = {}
    
    job_name_hh = job_hh.find('div',{'class':'vacancy-serp-item__info'}).getText()
    job_company_hh = job_hh.find('a',{'class':'bloko-link bloko-link_secondary'}).getText()
    job_city_hh = job_hh.find('span',{'class':'vacancy-serp-item__meta-info'}).getText().split()[0].replace(',', '')
      
    # Зарплата HH:
    job_salary_hh = job_hh.find('div',{'class':'vacancy-serp-item__sidebar'}).getText().replace('\xa0', '')
    min_salary_reg_hh = '^\d*\s\d*|^[от]+\s(\d*\s\d*)'
    max_salary_reg_hh = '-(\d*\s\d*)|^[до]+\s(\d*\s\d*)'
    currency_reg_hh = '([a-z]*)+$|([а-я]*)\.+$'
    min_salary_pattern_hh = re.compile(min_salary_reg_hh)
    max_salary_pattern_hh = re.compile(max_salary_reg_hh)
    currency_pattern_hh = re.compile(currency_reg_hh, re.I)
    min_salary_hh = min_salary_pattern_hh.search(job_salary_hh)
    max_salary_hh = max_salary_pattern_hh.search(job_salary_hh)
    currency_hh = currency_pattern_hh.search(job_salary_hh)
    if min_salary_hh:
        min_salary_hh = min_salary_hh.group(0).replace('от', '').replace(' ', '')
    if max_salary_hh:
        max_salary_hh = max_salary_hh.group(0).replace('-', '').replace('до', '').replace(' ', '')
    if currency_hh:
        currency_hh = currency_hh.group(0).replace('.', '')
    #
    
    # Ссылка и ID для HH:
    job_link_hh = job_hh.find('a',{'class':'bloko-link HH-LinkModifier'})['href']
    job_id_reg_hh = '(\d+)'
    job_id_pattern_hh = re.compile(job_id_reg_hh)
    job_id_hh = job_id_pattern_hh.search(job_link_hh)
    if job_id_hh:
        job_id_hh = job_id_hh.group(0) + 'hh' 
    #
    
    job_data_hh['_id'] = job_id_hh
    job_data_hh['name'] = job_name_hh
    job_data_hh['company'] = job_company_hh
    job_data_hh['city'] = job_city_hh
    job_data_hh['min_salary'] = min_salary_hh
    job_data_hh['max_salary'] = max_salary_hh
    job_data_hh['currency'] = currency_hh
    job_data_hh['link'] = job_link_hh
    job_data_hh['from'] = main_link_hh
        
    jobs_hh.append(job_data_hh)

In [10]:
add_jobs_hh_to_db(jobs_hh)

True

#### SuperJob

In [11]:
main_link_sj = 'https://russia.superjob.ru'
header_sj = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.109 Safari/537.36',
          'Accept':'*/*'}
params_sj = {'keywords':'Data'}

In [12]:
response_sj = rq.get(main_link_sj + '/vacancy/search', headers=header_sj, params=params_sj).text
soup_sj = bs(response_sj, 'lxml')
vacancy_block_sj = soup_sj.find('div',{'class':'_1ID8B'})
vacancy_list_sj = vacancy_block_sj.find_all('div',{'class':'iJCa5 f-test-vacancy-item _1fma_ _1JhPh _2gFpt _1znz6 _2nteL'})

In [13]:
pages_sj = []
while True:
    if soup_sj.find('a',{'class':'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-Dalshe'}):
        next_page_sj = main_link_sj + soup_sj.find('a',{'class':'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-Dalshe'})['href']
    else:
        break
    pages_sj.append(next_page_sj)
    response_sj = rq.get(next_page_sj, headers=header_sj).text
    soup_sj = bs(response_sj, 'lxml')
    vacancy_block_sj_ = soup_sj.find('div',{'class':'_1ID8B'})
    vacancy_list_sj_ = vacancy_block_sj_.find_all('div',{'class':'iJCa5 f-test-vacancy-item _1fma_ _1JhPh _2gFpt _1znz6 _2nteL'})
pprint(pages_sj)

['https://russia.superjob.ru/vacancy/search/?keywords=Data&page=2',
 'https://russia.superjob.ru/vacancy/search/?keywords=Data&page=3',
 'https://russia.superjob.ru/vacancy/search/?keywords=Data&page=4']


In [14]:
jobs_sj = []

In [15]:
# Функция добавления вакансий SJ в базу данных

def add_jobs_sj_to_db(jobs_sj):
    
    jobs.insert_many(jobs_sj)

    return True

In [16]:
for job_sj in vacancy_list_sj + vacancy_list_sj_:
    
    job_data_sj = {}
    
    job_name_sj = job_sj.find('div',{'class':'_3mfro PlM3e _2JVkc _3LJqf'}).getText()
    job_company_sj = job_sj.find('span',{'class':'_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _2VHxz _15msI'}).getText()
    job_city_sj = job_sj.find('span',{'class':'_3mfro f-test-text-company-item-location _9fXTd _2JVkc _2VHxz'}).getText().split()[-1]
    
    # Зарплата SJ:
    job_salary_sj = job_sj.find('span',{'class':'_1OuF_ _1qw9T f-test-text-company-item-salary'}).getText().replace('\xa0', '').replace('/месяц', '')
    min_salary_reg_sj = '-(\d*\d*)|^[от]+(\d*\d*)'
    max_salary_reg_sj = '-(\d*\d*)|^[до]+(\d*\d*)'
    currency_reg_sj = '([a-z]*)+$|([а-я]*)\.+$'
    min_salary_pattern_sj = re.compile(min_salary_reg_sj)
    max_salary_pattern_sj = re.compile(max_salary_reg_sj)
    currency_pattern_sj = re.compile(currency_reg_sj, re.I)
    min_salary_sj = min_salary_pattern_sj.search(job_salary_sj)
    max_salary_sj = max_salary_pattern_sj.search(job_salary_sj)
    currency_sj = currency_pattern_sj.search(job_salary_sj)
    if min_salary_sj:
        min_salary_sj = min_salary_sj.group(0).replace('от', '').replace('о', 'None').replace(' ', '')
    if max_salary_sj:
        max_salary_sj = max_salary_sj.group(0).replace('до', '').replace('о', 'None').replace(' ', '')
    if currency_sj:
        currency_sj = currency_sj.group(0).replace('.', '')
    #
    
    # Ссылка и ID для SJ:
    job_link_sj = main_link_sj + job_sj.find('a',{'class':'icMQ_'})['href']
    job_id_reg_sj = '(\d+\.)'
    job_id_pattern_sj = re.compile(job_id_reg_sj)
    job_id_sj = job_id_pattern_sj.search(job_link_sj)
    if job_id_sj:
        job_id_sj = job_id_sj.group(0).replace('.', '') + 'sj'
    #
        
    job_data_sj['_id'] = job_id_sj 
    job_data_sj['name'] = job_name_sj
    job_data_sj['company'] = job_company_sj
    job_data_sj['city'] = job_city_sj
    job_data_sj['min_salary'] = min_salary_sj
    job_data_sj['max_salary'] = max_salary_sj
    job_data_sj['currency'] = currency_sj
    job_data_sj['link'] = job_link_sj
    job_data_sj['from'] = main_link_sj
    
    jobs_sj.append(job_data_sj)

In [17]:
add_jobs_sj_to_db(jobs_sj)

True

#### Задание 2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы. Поиск по двум полям (мин и макс зарплату)

In [18]:
def higher_salary(salary):
    for job in jobs.find( {'$or': [ {'min_salary': {'$gt': salary}}, {'max_salary': {'$gt': salary}} ] } ):
        pprint(job)

In [19]:
salary_input = input('Введите уровень зарплаты: ')
higher_salary(salary_input)

Введите уровень зарплаты: 41000
{'_id': '37813468hh',
 'city': 'Казань',
 'company': ' Город Иннополис',
 'currency': 'руб',
 'from': 'https://spb.hh.ru',
 'link': 'https://spb.hh.ru/vacancy/37813468?query=Data%20scientist',
 'max_salary': None,
 'min_salary': '70000',
 'name': 'Data Scientist'}
{'_id': '37622381hh',
 'city': 'Москва',
 'company': ' MAXIMUM EDUCATION',
 'currency': 'руб',
 'from': 'https://spb.hh.ru',
 'link': 'https://spb.hh.ru/vacancy/37622381?query=Data%20scientist',
 'max_salary': None,
 'min_salary': '65000',
 'name': 'IT Recruiter'}
{'_id': '37469880hh',
 'city': 'Томск',
 'company': ' WiseTech',
 'currency': 'руб',
 'from': 'https://spb.hh.ru',
 'link': 'https://spb.hh.ru/vacancy/37469880?query=Data%20scientist',
 'max_salary': '70000',
 'min_salary': None,
 'name': 'Стажер data scientist в финтех компании'}
{'_id': '34064156sj',
 'city': 'Молодежная',
 'company': '7ЦВЕТОВ и 7ЦВЕТОВ Декор',
 'currency': 'руб',
 'from': 'https://russia.superjob.ru',
 'link': 'htt

#### Задание 3. Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [20]:
def add_new_jobs_hh_to_db(jobs_hh):
    
    for hh in jobs_hh:
        try:
            jobs.insert_one(hh)
        except errors.DuplicateKeyError:
            pass
            return False # Если вакансия с HH уже есть
        else:
            return True # Если вакансии с HH не было

In [21]:
def add_new_jobs_sj_to_db(jobs_sj):
    
    for sj in jobs_sj:
        try:
            jobs.insert_one(sj)
        except errors.DuplicateKeyError:
            pass
            return False # Если вакансия с SJ уже есть
        else:
            return True # Если вакансии с SJ не было